In [18]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pyproj
import shapely
from typing import List

In [19]:
# Set notebook display options
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [20]:
def geod2utm(row):
    '''   Convert geodetic coordinates to UTM   '''
    zn = '16T'
    datum = 'WGS84'
    lat = row['lat']
    lon = row['lon']
        
    p = pyproj.Proj(proj='utm', zone=zn, ellps=datum)
    
    X, Y = p(lon, lat)
    
    return pd.Series({'UTMx': X, 'UTMy': Y})

In [21]:
def make_utm_points(row):
    UTMx = row['UTMx']
    UTMy = row['UTMy']
    UTMPoint = shapely.geometry.Point(UTMx, UTMy)
    return UTMPoint

# Read in summary data from the 2010 census

In [22]:
census_summary_df = pd.read_csv('data/2010-census-summary.csv')

In [23]:
for i in range(67,129):
    del census_summary_df['Unnamed: {0}'.format(i)]

# Read in shapes of community areas

In [24]:
communities = gpd.read_file('data/community-areas.geojson')

In [25]:
census_summary_df.to_pickle('data/census_summary-cleaned.pkl')

In [26]:
communities_df = pd.read_json('data/community-areas.geojson')

In [27]:
community_names: List[str] = []
community_area_numbers: List[int] = []
community_geometries: List[shapely.geometry.Polygon] = []
community_modeled: pd.DataFrame = pd.DataFrame([{'community name': 'DELETE THIS',
                                                 'community number': 0,
                                                 'UTMGeometry': 'DELETE THIS'
                                                }])

In [28]:
del community_modeled
# community_modeled_dict: dict = {}
community_names: list = []
community_area_numbers: list = []
community_geometries:list = []
community_area_sizes: list = []
for feature in communities_df['features']:
    community_poly_points: list = []
    community_names.append(feature['properties']['community'])
    community_area_numbers.append(feature['properties']['area_numbe'])
    for point in feature['geometry']['coordinates'][0][0]:
        community_poly_points.append((geod2utm(pd.Series({'lon': point[0], 'lat': point[1]}))))
    community_polygon = shapely.geometry.Polygon(community_poly_points)
    community_geometries.append(community_polygon)
    community_area_sizes.append(community_polygon.area)

In [29]:
community_modeled = pd.DataFrame({'community name': community_names,
                                  'community number': community_area_numbers,
                                  'UTMGeometry': community_geometries,
                                  'community area': community_area_sizes
                                 },
                                 index=range(len(community_names)))

In [30]:
community_modeled['community number'] = community_modeled['community number'].astype('int64')

In [32]:
community_modeled.to_pickle('data/community_modeled.pkl')

In [33]:
census_detail_df = pd.read_csv('data/il000012010.dp')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Columns: 191 entries, DPST to 2.38
dtypes: float64(9), int64(180), object(2)
memory usage: 4.6+ MB


In [52]:
census_tracts_df = gpd.read_file('data/CensusTracts-2010.geojson')
census_tracts_df['geoid10'] = census_tracts_df['geoid10'].astype('int64')

,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10,geometry
0,17,8424,44,Census Tract 8424,44,17031842400,,842400,031,(POLYGON ((-87.62404799998049 41.7302169999839...
1,17,8403,59,Census Tract 8403,59,17031840300,,840300,031,(POLYGON ((-87.6860799999848 41.82295600001154...
2,17,8411,34,Census Tract 8411,34,17031841100,,841100,031,(POLYGON ((-87.62934700001182 41.8527970000265...
3,17,8412,31,Census Tract 8412,31,17031841200,,841200,031,(POLYGON ((-87.68813499997718 41.8556909999909...
4,17,8390,32,Census Tract 8390,32,17031839000,,839000,031,(POLYGON ((-87.63312200003458 41.8744880000269...


In [81]:
age_sex_race_relationship_housing_df = pd.read_csv('data/DEC_10_SF1_SF1DP1_with_ann.csv')

In [82]:
del age_sex_race_relationship_housing_df['Id']
del age_sex_race_relationship_housing_df['Number; RACE - Total population']
del age_sex_race_relationship_housing_df['Percent; RACE - Total population']
del age_sex_race_relationship_housing_df['Number; RELATIONSHIP - Total population']
del age_sex_race_relationship_housing_df['Percent; RELATIONSHIP - Total population']
del age_sex_race_relationship_housing_df['Number; HOUSEHOLDS BY TYPE - Total households']
del age_sex_race_relationship_housing_df['Percent; HOUSEHOLDS BY TYPE - Total households']
del age_sex_race_relationship_housing_df['Percent; HOUSEHOLDS BY TYPE - Total households - Average household size']
del age_sex_race_relationship_housing_df['Percent; HOUSEHOLDS BY TYPE - Total households - Average family size [7]']
del age_sex_race_relationship_housing_df['Percent; HOUSING OCCUPANCY - Total housing units - Homeowner vacancy rate (percent) [8]']
del age_sex_race_relationship_housing_df['Percent; HOUSING OCCUPANCY - Total housing units - Rental vacancy rate (percent) [9]']
del age_sex_race_relationship_housing_df['Percent; HOUSING TENURE - Occupied housing units - Owner-occupied housing units - Population in owner-occupied housing units']
del age_sex_race_relationship_housing_df['Percent; HOUSING TENURE - Occupied housing units - Owner-occupied housing units - Average household size of owner-occupied units']
del age_sex_race_relationship_housing_df['Percent; HOUSING TENURE - Occupied housing units - Renter-occupied housing units - Population in renter-occupied housing units']
del age_sex_race_relationship_housing_df['Percent; HOUSING TENURE - Occupied housing units - Renter-occupied housing units - Average household size of renter-occupied units']

In [83]:
age_sex_race_relationship_housing_df.rename(columns={'Id2': 'tract id',
                                                     'Geography': 'tract description',
                                                     'Number; SEX AND AGE - Total population': 'total population - number',
                                                     'Percent; SEX AND AGE - Total population': 'total population - percent'
                                                    }, inplace=True)

,tract id,tract description,total population - number,total population - percent,Number; SEX AND AGE - Total population - Under 5 years,Percent; SEX AND AGE - Total population - Under 5 years,Number; SEX AND AGE - Total population - 5 to 9 years,Percent; SEX AND AGE - Total population - 5 to 9 years,Number; SEX AND AGE - Total population - 10 to 14 years,Percent; SEX AND AGE - Total population - 10 to 14 years,Number; SEX AND AGE - Total population - 15 to 19 years,Percent; SEX AND AGE - Total population - 15 to 19 years,Number; SEX AND AGE - Total population - 20 to 24 years,Percent; SEX AND AGE - Total population - 20 to 24 years,Number; SEX AND AGE - Total population - 25 to 29 years,Percent; SEX AND AGE - Total population - 25 to 29 years,Number; SEX AND AGE - Total population - 30 to 34 years,Percent; SEX AND AGE - Total population - 30 to 34 years,Number; SEX AND AGE - Total population - 35 to 39 years,Percent; SEX AND AGE - Total population - 35 to 39 years,Number; SEX AND AGE - Total population - 40 to 44 years,Percent; SEX AND AGE - Total population - 40 to 44 years,Number; SEX AND AGE - Total population - 45 to 49 years,Percent; SEX AND AGE - Total population - 45 to 49 years,Number; SEX AND AGE - Total population - 50 to 54 years,Percent; SEX AND AGE - Total population - 50 to 54 years,Number; SEX AND AGE - Total population - 55 to 59 years,Percent; SEX AND AGE - Total population - 55 to 59 years,Number; SEX AND AGE - Total population - 60 to 64 years,Percent; SEX AND AGE - Total population - 60 to 64 years,Number; SEX AND AGE - Total population - 65 to 69 years,Percent; SEX AND AGE - Total population - 65 to 69 years,Number; SEX AND AGE - Total population - 70 to 74 years,Percent; SEX AND AGE - Total population - 70 to 74 years,Number; SEX AND AGE - Total population - 75 to 79 years,Percent; SEX AND AGE - Total population - 75 to 79 years,Number; SEX AND AGE - Total population - 80 to 84 years,Percent; SEX AND AGE - Total population - 80 to 84 years,Number; SEX AND AGE - Total population - 85 years and over,Percent; SEX AND AGE - Total population - 85 years and over,Number; SEX AND AGE - Total population - Median age (years),Percent; SEX AND AGE - Total population - Median age (years),Number; SEX AND AGE - Total population - 16 years and over,Percent; SEX AND AGE - Total population - 16 years and over,Number; SEX AND AGE - Total population - 18 years and over,Percent; SEX AND AGE - Total population - 18 years and over,Number; SEX AND AGE - Total population - 21 years and over,Percent; SEX AND AGE - Total population - 21 years and over,Number; SEX AND AGE - Total population - 62 years and over,Percent; SEX AND AGE - Total population - 62 years and over,Number; SEX AND AGE - Total population - 65 years and over,Percent; SEX AND AGE - Total population - 65 years and over,Number; SEX AND AGE - Male population,Percent; SEX AND AGE - Male population,Number; SEX AND AGE - Male population - Under 5 years,Percent; SEX AND AGE - Male population - Under 5 years,Number; SEX AND AGE - Male population - 5 to 9 years,Percent; SEX AND AGE - Male population - 5 to 9 years,Number; SEX AND AGE - Male population - 10 to 14 years,Percent; SEX AND AGE - Male population - 10 to 14 years,Number; SEX AND AGE - Male population - 15 to 19 years,Percent; SEX AND AGE - Male population - 15 to 19 years,Number; SEX AND AGE - Male population - 20 to 24 years,Percent; SEX AND AGE - Male population - 20 to 24 years,Number; SEX AND AGE - Male population - 25 to 29 years,Percent; SEX AND AGE - Male population - 25 to 29 years,Number; SEX AND AGE - Male population - 30 to 34 years,Percent; SEX AND AGE - Male population - 30 to 34 years,Number; SEX AND AGE - Male population - 35 to 39 years,Percent; SEX AND AGE - Male population - 35 to 39 years,Number; SEX AND AGE - Male population - 40 to 44 years,Percent; SEX AND AGE - Male population - 40 to 44 years,Number; SEX AND AGE - Male population - 45 to 49 years,Percent; SEX AND AGE - Male population - 45 

In [86]:
chicago_only_census_df = pd.merge(census_tracts_df,
                                  age_sex_race_relationship_housing_df,
                                  left_on='geoid10',
                                  right_on='tract id'
                                 )

,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10,geometry,tract id,tract description,total population - number,total population - percent,Number; SEX AND AGE - Total population - Under 5 years,Percent; SEX AND AGE - Total population - Under 5 years,Number; SEX AND AGE - Total population - 5 to 9 years,Percent; SEX AND AGE - Total population - 5 to 9 years,Number; SEX AND AGE - Total population - 10 to 14 years,Percent; SEX AND AGE - Total population - 10 to 14 years,Number; SEX AND AGE - Total population - 15 to 19 years,Percent; SEX AND AGE - Total population - 15 to 19 years,Number; SEX AND AGE - Total population - 20 to 24 years,Percent; SEX AND AGE - Total population - 20 to 24 years,Number; SEX AND AGE - Total population - 25 to 29 years,Percent; SEX AND AGE - Total population - 25 to 29 years,Number; SEX AND AGE - Total population - 30 to 34 years,Percent; SEX AND AGE - Total population - 30 to 34 years,Number; SEX AND AGE - Total population - 35 to 39 years,Percent; SEX AND AGE - Total population - 35 to 39 years,Number; SEX AND AGE - Total population - 40 to 44 years,Percent; SEX AND AGE - Total population - 40 to 44 years,Number; SEX AND AGE - Total population - 45 to 49 years,Percent; SEX AND AGE - Total population - 45 to 49 years,Number; SEX AND AGE - Total population - 50 to 54 years,Percent; SEX AND AGE - Total population - 50 to 54 years,Number; SEX AND AGE - Total population - 55 to 59 years,Percent; SEX AND AGE - Total population - 55 to 59 years,Number; SEX AND AGE - Total population - 60 to 64 years,Percent; SEX AND AGE - Total population - 60 to 64 years,Number; SEX AND AGE - Total population - 65 to 69 years,Percent; SEX AND AGE - Total population - 65 to 69 years,Number; SEX AND AGE - Total population - 70 to 74 years,Percent; SEX AND AGE - Total population - 70 to 74 years,Number; SEX AND AGE - Total population - 75 to 79 years,Percent; SEX AND AGE - Total population - 75 to 79 years,Number; SEX AND AGE - Total population - 80 to 84 years,Percent; SEX AND AGE - Total population - 80 to 84 years,Number; SEX AND AGE - Total population - 85 years and over,Percent; SEX AND AGE - Total population - 85 years and over,Number; SEX AND AGE - Total population - Median age (years),Percent; SEX AND AGE - Total population - Median age (years),Number; SEX AND AGE - Total population - 16 years and over,Percent; SEX AND AGE - Total population - 16 years and over,Number; SEX AND AGE - Total population - 18 years and over,Percent; SEX AND AGE - Total population - 18 years and over,Number; SEX AND AGE - Total population - 21 years and over,Percent; SEX AND AGE - Total population - 21 years and over,Number; SEX AND AGE - Total population - 62 years and over,Percent; SEX AND AGE - Total population - 62 years and over,Number; SEX AND AGE - Total population - 65 years and over,Percent; SEX AND AGE - Total population - 65 years and over,Number; SEX AND AGE - Male population,Percent; SEX AND AGE - Male population,Number; SEX AND AGE - Male population - Under 5 years,Percent; SEX AND AGE - Male population - Under 5 years,Number; SEX AND AGE - Male population - 5 to 9 years,Percent; SEX AND AGE - Male population - 5 to 9 years,Number; SEX AND AGE - Male population - 10 to 14 years,Percent; SEX AND AGE - Male population - 10 to 14 years,Number; SEX AND AGE - Male population - 15 to 19 years,Percent; SEX AND AGE - Male population - 15 to 19 years,Number; SEX AND AGE - Male population - 20 to 24 years,Percent; SEX AND AGE - Male population - 20 to 24 years,Number; SEX AND AGE - Male population - 25 to 29 years,Percent; SEX AND AGE - Male population - 25 to 29 years,Number; SEX AND AGE - Male population - 30 to 34 years,Percent; SEX AND AGE - Male population - 30 to 34 years,Number; SEX AND AGE - Male population - 35 to 39 years,Percent; SEX AND AGE - Male population - 35 to 39 years,Number; SEX AND AGE - Male population - 40 to 44 years,Percent; SEX AND AGE - Male population - 40 to 44 years,Number;

In [87]:
del chicago_only_census_df['statefp10']
del chicago_only_census_df['name10']
del chicago_only_census_df['commarea_n']
del chicago_only_census_df['namelsad10']
del chicago_only_census_df['commarea']
del chicago_only_census_df['geoid10']
del chicago_only_census_df['notes']
del chicago_only_census_df['tractce10']
del chicago_only_census_df['countyfp10']

,geometry,tract id,tract description,total population - number,total population - percent,Number; SEX AND AGE - Total population - Under 5 years,Percent; SEX AND AGE - Total population - Under 5 years,Number; SEX AND AGE - Total population - 5 to 9 years,Percent; SEX AND AGE - Total population - 5 to 9 years,Number; SEX AND AGE - Total population - 10 to 14 years,Percent; SEX AND AGE - Total population - 10 to 14 years,Number; SEX AND AGE - Total population - 15 to 19 years,Percent; SEX AND AGE - Total population - 15 to 19 years,Number; SEX AND AGE - Total population - 20 to 24 years,Percent; SEX AND AGE - Total population - 20 to 24 years,Number; SEX AND AGE - Total population - 25 to 29 years,Percent; SEX AND AGE - Total population - 25 to 29 years,Number; SEX AND AGE - Total population - 30 to 34 years,Percent; SEX AND AGE - Total population - 30 to 34 years,Number; SEX AND AGE - Total population - 35 to 39 years,Percent; SEX AND AGE - Total population - 35 to 39 years,Number; SEX AND AGE - Total population - 40 to 44 years,Percent; SEX AND AGE - Total population - 40 to 44 years,Number; SEX AND AGE - Total population - 45 to 49 years,Percent; SEX AND AGE - Total population - 45 to 49 years,Number; SEX AND AGE - Total population - 50 to 54 years,Percent; SEX AND AGE - Total population - 50 to 54 years,Number; SEX AND AGE - Total population - 55 to 59 years,Percent; SEX AND AGE - Total population - 55 to 59 years,Number; SEX AND AGE - Total population - 60 to 64 years,Percent; SEX AND AGE - Total population - 60 to 64 years,Number; SEX AND AGE - Total population - 65 to 69 years,Percent; SEX AND AGE - Total population - 65 to 69 years,Number; SEX AND AGE - Total population - 70 to 74 years,Percent; SEX AND AGE - Total population - 70 to 74 years,Number; SEX AND AGE - Total population - 75 to 79 years,Percent; SEX AND AGE - Total population - 75 to 79 years,Number; SEX AND AGE - Total population - 80 to 84 years,Percent; SEX AND AGE - Total population - 80 to 84 years,Number; SEX AND AGE - Total population - 85 years and over,Percent; SEX AND AGE - Total population - 85 years and over,Number; SEX AND AGE - Total population - Median age (years),Percent; SEX AND AGE - Total population - Median age (years),Number; SEX AND AGE - Total population - 16 years and over,Percent; SEX AND AGE - Total population - 16 years and over,Number; SEX AND AGE - Total population - 18 years and over,Percent; SEX AND AGE - Total population - 18 years and over,Number; SEX AND AGE - Total population - 21 years and over,Percent; SEX AND AGE - Total population - 21 years and over,Number; SEX AND AGE - Total population - 62 years and over,Percent; SEX AND AGE - Total population - 62 years and over,Number; SEX AND AGE - Total population - 65 years and over,Percent; SEX AND AGE - Total population - 65 years and over,Number; SEX AND AGE - Male population,Percent; SEX AND AGE - Male population,Number; SEX AND AGE - Male population - Under 5 years,Percent; SEX AND AGE - Male population - Under 5 years,Number; SEX AND AGE - Male population - 5 to 9 years,Percent; SEX AND AGE - Male population - 5 to 9 years,Number; SEX AND AGE - Male population - 10 to 14 years,Percent; SEX AND AGE - Male population - 10 to 14 years,Number; SEX AND AGE - Male population - 15 to 19 years,Percent; SEX AND AGE - Male population - 15 to 19 years,Number; SEX AND AGE - Male population - 20 to 24 years,Percent; SEX AND AGE - Male population - 20 to 24 years,Number; SEX AND AGE - Male population - 25 to 29 years,Percent; SEX AND AGE - Male population - 25 to 29 years,Number; SEX AND AGE - Male population - 30 to 34 years,Percent; SEX AND AGE - Male population - 30 to 34 years,Number; SEX AND AGE - Male population - 35 to 39 years,Percent; SEX AND AGE - Male population - 35 to 39 years,Number; SEX AND AGE - Male population - 40 to 44 years,Percent; SEX AND AGE - Male population - 40 to 44 years,Number; SEX AND AGE - Male population - 45 to 49 years,Percent; SEX AND AGE - Male populat

In [89]:
chicago_only_census_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 801 entries, 0 to 800
Columns: 361 entries, geometry to Number; HOUSING TENURE - Occupied housing units - Renter-occupied housing units - Average household size of renter-occupied units
dtypes: float64(7), int64(171), object(183)
memory usage: 2.2+ MB


In [88]:
chicago_only_census_df.to_pickle('data/census-2010-chicago-age-sex-race-relationship-housing-transformed.pkl')